In [23]:
import os
import librosa
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import LabelEncoder
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
import soundfile as sf


In [24]:
DATA_PATH_TRAIN = './dataset/train'
DATA_PATH_TEST = './dataset/test'

SAMPLE_RATE = 22050
MFCC_COUNT = 40
TOP_DB = 20


In [25]:
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift

def remove_silence(audio, sr, top_db=TOP_DB):
    y_trimmed, _ = librosa.effects.trim(audio, top_db=top_db)
    return y_trimmed

def augment_audio(audio, sr):
    augmenter = Compose([
        AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.015, p=0.3),
        TimeStretch(min_rate=0.8, max_rate=1.25, p=0.3),
        PitchShift(min_semitones=-4, max_semitones=4, p=0.3),
        Shift(min_fraction=-0.5, max_fraction=0.5, p=0.3),
    ])
    audio_augmented = augmenter(samples=audio, sample_rate=sr)
    return audio_augmented

def extract_features(audio, sr):
    mfccs = librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=MFCC_COUNT)
    chroma_stft = librosa.feature.chroma_stft(y=audio, sr=sr)
    spectral_contrast = librosa.feature.spectral_contrast(y=audio, sr=sr)
    tonnetz = librosa.feature.tonnetz(y=audio, sr=sr)
    return np.concatenate((np.mean(mfccs.T, axis=0), 
                           np.mean(chroma_stft.T, axis=0),
                           np.mean(spectral_contrast.T, axis=0),
                           np.mean(tonnetz.T, axis=0)))


In [26]:
def load_data(data_path, augment=False):
    features = []
    labels = []
    for label in ["human", "other"]:
        folder_path = os.path.join(data_path, label)
        for filename in os.listdir(folder_path):
            if not (filename.endswith('.mp3') or filename.endswith('.wav') or filename.endswith('.flac')):
                continue
            file_path = os.path.join(folder_path, filename)
            audio, _ = librosa.load(file_path, sr=SAMPLE_RATE, mono=True)
            audio = remove_silence(audio, SAMPLE_RATE)
            if augment:
                audio = augment_audio(audio, SAMPLE_RATE)
            feature = extract_features(audio, SAMPLE_RATE)
            features.append(feature)
            labels.append(label)
    return np.array(features), np.array(labels)


In [27]:
X, y = load_data(DATA_PATH_TRAIN, augment=True)

le = LabelEncoder()
y = le.fit_transform(y)


c:\Users\KB\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=984
  warnings.warn(
c:\Users\KB\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=760
  warnings.warn(
c:\Users\KB\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=832
  warnings.warn(
c:\Users\KB\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=984
  warnings.warn(
c:\Users\KB\AppData\Local\Programs\Python\Python310\lib\site-packages\librosa\core\spectrum.py:256: UserWarning: n_fft=1024 is too large for input signal of length=944
  warnings.warn(
c:\Users\KB\AppData\Local\Programs\Python\Python310\lib\site-packages\libro

In [33]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


In [34]:
model = Sequential([
    Dense(100, activation='relu', kernel_initializer="he_normal", input_shape=(X_train.shape[1],)),
    Dense(50, activation='relu', kernel_initializer="he_normal"),
    Dense(1, activation='sigmoid'),
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [35]:
es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=50, batch_size=32, callbacks=[es])


Epoch 1/50
216/216 [==============================] - 2s 4ms/step - loss: 0.5879 - accuracy: 0.8718 - val_loss: 0.3465 - val_accuracy: 0.8986
Epoch 2/50
216/216 [==============================] - 1s 3ms/step - loss: 0.2755 - accuracy: 0.9156 - val_loss: 0.2108 - val_accuracy: 0.9363
Epoch 3/50
216/216 [==============================] - 1s 3ms/step - loss: 0.2081 - accuracy: 0.9312 - val_loss: 0.1873 - val_accuracy: 0.9473
Epoch 4/50
216/216 [==============================] - 1s 3ms/step - loss: 0.1612 - accuracy: 0.9463 - val_loss: 0.2695 - val_accuracy: 0.9108
Epoch 5/50
216/216 [==============================] - 1s 3ms/step - loss: 0.1957 - accuracy: 0.9382 - val_loss: 0.2025 - val_accuracy: 0.9450
Epoch 6/50
216/216 [==============================] - 1s 3ms/step - loss: 0.1461 - accuracy: 0.9505 - val_loss: 0.1632 - val_accuracy: 0.9542
Epoch 7/50
216/216 [==============================] - 1s 3ms/step - loss: 0.1437 - accuracy: 0.9544 - val_loss: 0.1376 - val_accuracy: 0.9513
Epoch 

In [36]:
y_val_pred_prob = model.predict(X_val)
y_val_pred = (y_val_pred_prob > 0.5).astype("int32")

cm = confusion_matrix(y_val, y_val_pred)
print("Confusion Matrix:")
print(cm)

cr = classification_report(y_val, y_val_pred)
print("Classification Report:")
print(cr)

loss, accuracy = model.evaluate(X_val, y_val, verbose=0)
print("Model Accuracy: {:.2f}%".format(accuracy*100))
print("Model Loss: {:.2f}".format(loss))


54/54 [==============================] - 0s 3ms/step
Confusion Matrix:
[[ 309   14]
 [  38 1365]]
Classification Report:
              precision    recall  f1-score   support

           0       0.89      0.96      0.92       323
           1       0.99      0.97      0.98      1403

    accuracy                           0.97      1726
   macro avg       0.94      0.96      0.95      1726
weighted avg       0.97      0.97      0.97      1726

Model Accuracy: 96.99%
Model Loss: 0.11


In [37]:
def predict_audio_class(audio_file_path, model, le):
    audio, _ = librosa.load(audio_file_path, sr=SAMPLE_RATE, mono=True)
    audio = remove_silence(audio, SAMPLE_RATE)
    feature = extract_features(audio, SAMPLE_RATE)
    feature = np.expand_dims(feature, axis=0)  # because the model expects 2D array
    prediction_prob = model.predict(feature)
    prediction = (prediction_prob > 0.5).astype("int32")
    prediction_label = le.inverse_transform(prediction)[0]
    return prediction_label


In [43]:
audio_file_path = "./dataset/test/roy.wav"  # Adjust if necessary
prediction = predict_audio_class(audio_file_path, model, le)
print(f"The audio is predicted as: {prediction}")


1/1 [==============================] - 0s 24ms/step
The audio is predicted as: human


c:\Users\KB\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:155: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [46]:
import joblib

# Save your trained model
model.save("./model-human-speech-detection/model.h5")

# Save your label encoder
joblib.dump(le, "./model-human-speech-detection/label_encoder.joblib")

['./model-human-speech-detection/label_encoder.joblib']